In [1]:
import boto3
import pandas as pd
import datetime
import io

In [2]:
from nanoHUB.application import Application
application = Application.get_instance()

nanohub_metrics_db = application.new_db_engine('nanohub_metrics')

nanoHUB - Serving Students, Researchers & Instructors


In [3]:
bucket_name = 'nanoHUB-analytics'
endpoint_url = 'https://' + application.get_config_value('geddesapi.endpoint') + ':443'

boto_session = boto3.session.Session(
    aws_access_key_id = application.get_config_value('geddesapi.access_key'),
    aws_secret_access_key = application.get_config_value('geddesapi.secret_key'),
)

s3_client1 = boto_session.client(
    service_name='s3',
    endpoint_url = endpoint_url
) 


In [4]:
response = s3_client1.list_buckets()
print(response)

{'ResponseMetadata': {'RequestId': 'tx00000000000000001ebc9-0061913648-a6d52f-my-store', 'HostId': '', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 14 Nov 2021 16:16:08 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amz-request-id': 'tx00000000000000001ebc9-0061913648-a6d52f-my-store', 'strict-transport-security': 'max-age=15724800; includeSubDomains'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'db2.nanohub', 'CreationDate': datetime.datetime(2021, 11, 14, 7, 19, 33, 62000, tzinfo=tzlocal())}, {'Name': 'db2.nanohub-metrics.toolevents', 'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 930000, tzinfo=tzlocal())}, {'Name': 'db2.nanohub-metrics.toolstart', 'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 659000, tzinfo=tzlocal())}, {'Name': 'nanohub.raw', 'CreationDate': datetime.datetime(2021, 11, 14, 16, 3, 18, 337000, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'Praveen Saxena', 'ID': 'saxenap'}}


In [5]:
#bucket = s3_client1.create_bucket(Bucket='db2.nanohub-metrics.toolstart')
#bucket = s3_client1.create_bucket(Bucket='db2.nanohub-metrics.toolevents')
#bucket = s3_client1.create_bucket(Bucket='nanohub.raw')

#print(bucket)
display(s3_client1.list_buckets())

{'ResponseMetadata': {'RequestId': 'tx00000000000000001ebca-0061913649-a6d52f-my-store',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 14 Nov 2021 16:16:09 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amz-request-id': 'tx00000000000000001ebca-0061913649-a6d52f-my-store',
   'strict-transport-security': 'max-age=15724800; includeSubDomains'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'db2.nanohub',
   'CreationDate': datetime.datetime(2021, 11, 14, 7, 19, 33, 62000, tzinfo=tzlocal())},
  {'Name': 'db2.nanohub-metrics.toolevents',
   'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 930000, tzinfo=tzlocal())},
  {'Name': 'db2.nanohub-metrics.toolstart',
   'CreationDate': datetime.datetime(2021, 11, 14, 7, 32, 19, 659000, tzinfo=tzlocal())},
  {'Name': 'nanohub.raw',
   'CreationDate': datetime.datetime(2021, 11, 14, 16, 3, 18, 337000, tzinfo=tzlocal())}],
 'Owner': {'DisplayName

In [53]:
for key in s3_client1.list_objects(Bucket='nanohub.raw')['Contents']:
    print(key['Key'])
    obj = s3_client1.get_object(Bucket='nanohub.raw', Key=key['Key'])
    
    display(pd.read_parquet(io.BytesIO(obj['Body'].read())))
    

KeyError: 'Contents'

In [48]:
obj = s3_client1.get_object(Bucket='nanohub.raw', Key='nanohub_metrics/toolevents/2012-01-17.parquet.gzip')
display(pd.read_parquet(io.BytesIO(obj['Body'].read())))



NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: Unknown

In [52]:
for key in s3_client1.list_objects(Bucket='nanohub.raw')['Contents']:
    print(key['Key'])
    s3_client1.delete_object(Bucket='nanohub.raw', Key=key['Key'])

KeyError: 'Contents'

In [14]:
sql_query = '''
SELECT * FROM nanohub_metrics.toolevents 
    WHERE start >='2012-12-25 00:00:00'
    AND start <'2012-12-26 00:00:00'
;
'''


df = pd.read_sql_query(sql_query, nanohub_metrics_db)
display(df.head())
display(df.tail())
display(df.info())

,entryID,source,job,superjob,sessnum,event,start,finish,walltime,cputime,...,ip,host,domain,countryShort,countryLong,region,city,latitude,longitude,recStatus
0,6410036,from joblog,0,0,553089,application,2012-12-25 00:01:21,2012-12-25 00:01:21,0.0,1.664,...,218.2.216.14,unknown,unknown,CN,China,Jiangsu,Nanjing,32.061670,118.777780,1
1,6482224,from joblog,2705861,0,546612,simulation,2012-12-25 00:01:47,2012-12-25 00:01:47,0.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2,6482225,from joblog,2705861,0,546612,[waiting],2012-12-25 00:01:47,2012-12-25 00:02:51,64.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
3,6409771,from joblog,0,0,553090,application,2012-12-25 00:02:18,2012-12-25 02:18:55,8197.0,0.992,...,85.90.101.194,pulsarnpp.ru,pulsarnpp.ru,RU,Russian Federation,Moskva,Moscow,55.752220,37.615560,1
4,6482226,from joblog,2705862,0,546612,simulation,2012-12-25 00:02:19,2012-12-25 00:02:20,1.0,0.020,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1


,entryID,source,job,superjob,sessnum,event,start,finish,walltime,cputime,...,ip,host,domain,countryShort,countryLong,region,city,latitude,longitude,recStatus
2863,6484803,from joblog,2707453,0,546612,simulation,2012-12-25 23:58:48,2012-12-25 23:58:48,0.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2864,6484804,from joblog,2707453,0,546612,[waiting],2012-12-25 23:58:48,2012-12-26 00:12:47,839.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2865,6484805,from joblog,2707454,0,546612,simulation,2012-12-25 23:58:57,2012-12-25 23:58:57,0.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2866,6484806,from joblog,2707454,0,546612,[waiting],2012-12-25 23:58:57,2012-12-26 00:02:43,226.0,0.000,...,128.46.16.15,springtide.hubzero.org,hubzero.org,US,United States,Indiana,West Lafayette,40.368891,-86.877431,1
2867,6410042,from joblog,0,0,553171,application,2012-12-25 23:59:13,2012-12-26 00:24:49,1536.0,1.319,...,120.194.82.142,unknown,unknown,CN,China,Henan,Zhumadian,32.979440,114.029440,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2868 entries, 0 to 2867
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   entryID       2868 non-null   int64         
 1   source        2868 non-null   object        
 2   job           2868 non-null   int64         
 3   superjob      2868 non-null   int64         
 4   sessnum       2868 non-null   int64         
 5   event         2868 non-null   object        
 6   start         2868 non-null   datetime64[ns]
 7   finish        2868 non-null   datetime64[ns]
 8   walltime      2868 non-null   float64       
 9   cputime       2868 non-null   float64       
 10  viewtime      2868 non-null   float64       
 11  user          2868 non-null   object        
 12  tool          2868 non-null   object        
 13  protocol      2868 non-null   int64         
 14  success       2868 non-null   int64         
 15  status        2868 non-null   int64   

None

In [ ]:
start = datetime.date(2012,12,25)
end = datetime.date.today()
end = datetime.date(2012,12,26)



#while start < end:
nextday = start + datetime.timedelta(days = 1)
print(start, nextday, end)
sql_query_daily = "select * from nanohub_metrics.toolevents where start >= '%s 00:00:00' and start < '%s 00:00:00'" % (start, nextday)
print(sql_query_daily)

df = pd.read_sql_query(sql_query_daily, nanohub_metrics_db)
display(df.head())
display(df.tail())
display(df.info())

start = nextday